In [ ]:
%pip install python-dotenv
%pip install langchain
%pip install openai
%pip install docarray
%pip install tiktoken

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
llm_model ="gpt-3.5-turbo"



In [6]:
from langchain.document_loaders import CSVLoader
file_path = r"C:\Users\write\Downloads\OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file_path)
docs = loader.load()

In [19]:
from langchain.indexes import VectorstoreIndexCreator

from langchain.vectorstores import DocArrayInMemorySearch
index = VectorstoreIndexCreator(
    vectorstore_cls = DocArrayInMemorySearch
).from_loaders([loader])

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
llm= ChatOpenAI(model = llm_model)

qa = RetrievalQA.from_chain_type(
    chain_type = "stuff",
    llm= llm,
    retriever = index.vectorstore.as_retriever(),
    verbose = True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)


In [22]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [23]:
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model= llm_model))

new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in docs[:5]]
)

examples += new_examples

qa.run(examples[0]["query"])

c:\Users\write\anaconda3\envs\langchain_for_llm_course\lib\site-packages\langchain\chains\llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set does have side pockets.'